In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import re
import json
import time
import datetime
from tqdm import tqdm

import threading

In [2]:
from crawler import APIClient

In [3]:
api = APIClient()
print(api)

Chrome Webscraper <selenium.webdriver.chrome.webdriver.WebDriver (session="6cde7e7cb0cc4cd05c259311450ed9f0")> (/Users/dillonmarquard/.wdm/drivers/chromedriver/mac64/112.0.5615/chromedriver)


In [ ]:
api.get_stock_data('tsla','2023-01-24',15)

In [4]:
class apiThreadSlave(threading.Thread):
   def __init__(self, thread_id, tag, dates, num_links):
      threading.Thread.__init__(self)
      self.thread_id = thread_id
      self.tag = tag
      self.dates = dates
      self.num_links = num_links
      self.api = APIClient()
   def run(self):
      for i, date in enumerate(self.dates):
         tmpdata = self.api.get_stock_data(self.tag, date,self.num_links)
         if tmpdata.shape[0] > 0:
            tmpdata.to_csv(r'data/{}/{}.csv'.format(self.tag,date))
         print('Thread ID:',self.thread_id,'Link #',i)

In [5]:
#@title Multithreading
MAX_THREADS = 24
TAG = 'tsla'

dt_rng = pd.date_range(start="2022-01-01", end="2022-12-31").date
n = len(dt_rng) // MAX_THREADS # number of dates per thread
thread_dt_rng = [dt_rng[x * n:(x + 1) * n] for x in range((len(dt_rng) + n - 1) // n )]

threads = []

for i, dates in enumerate(thread_dt_rng):
    tmp_thread = apiThreadSlave(i, TAG, dates, 25)
    tmp_thread.start()
    threads.append(tmp_thread)

for t in threads:
    t.join()

# TESTING

In [ ]:
#url = "https://www.bloomberg.com/news/articles/2023-02-17/summers-says-too-soon-to-call-for-march-50-basis-point-fed-hike?srnd=premium"
#url = "https://www.bloomberg.com/news/articles/2023-02-18/cars-tires-textile-factories-have-shut-in-crisis-hit-pakistan?srnd=industries-v2"

testlinks = api.get_google_links('tsla','2022-02-01',25)
testlinks

In [ ]:
res = api.get_article_data(testlinks[4])
res

In [ ]:
html = api.get_html_from_url(testlinks[0])

In [ ]:
soup = BeautifulSoup(html, 'html.parser')
text = ''
for tmp in [tmp.text for tmp in soup.find_all(['a','p','h','h1','h2'])]:
    if len(tmp) > 25 and re.findall('Click here',tmp) == [] and re.findall('This Simple Trick',tmp) == []:
        text += tmp + ' '
text = re.sub('\s+',' ',text.replace('\n',' ').replace('\xa0',' ').replace('\'','’').replace('   ',' ').strip())
text